In [ ]:
archivo_sincronizado = "The.Dark.Knight.Rises.2012.720p.BluRay.x264.YIFY.srt"  # Cambia esto por la ruta a tu archivo sincronizado .srt
archivo_desincronizado = "The.Dark.Knight.Rises.2012.1080p.BluRay.x265.srt"  # Cambia esto por la ruta a tu archivo desincronizado .srt
archivo_salida = "archivo_synced.srt"  # Nombre del archivo de salida

In [ ]:
import pandas as pd

# Función para convertir el Timestamp a segundos desde medianoche
def timestamp_a_segundos(tiempo):
    return tiempo.hour * 3600 + tiempo.minute * 60 + tiempo.second + tiempo.microsecond / 1e6

# Función para leer los archivos .srt y convertirlos a un diccionario con tiempos y texto
def procesar_srt(archivo_srt):
    diccionario = {}
    with open(archivo_srt, 'r', encoding='latin-1') as file:
        lines = file.readlines()

    n = len(lines)
    i = 0
    while i < n:
        if lines[i].strip().isdigit():
            nro_linea = int(lines[i].strip())
            tiempos = lines[i + 1].strip().split(' --> ')
            o_tiempo_str = tiempos[0]
            f_tiempo_str = tiempos[1]
            texto = lines[i + 2].strip()
            
            # Convertir tiempos a formato Timestamp
            o_tiempo = pd.to_datetime(o_tiempo_str, format='%H:%M:%S,%f')
            f_tiempo = pd.to_datetime(f_tiempo_str, format='%H:%M:%S,%f')
            
            diccionario[nro_linea] = {
                'o_tiempo': o_tiempo,
                'f_tiempo': f_tiempo,
                'texto': texto
            }
            
            i += 4  # Saltar a la siguiente sección (número, tiempos, texto y línea vacía)
        else:
            i += 1
    return diccionario

# Función principal para sincronizar los archivos .srt
def sincronizar_srt(archivo_sync, archivo_desync):
    # Procesar ambos archivos
    diccionario_sync = procesar_srt(archivo_sync)
    diccionario_desync = procesar_srt(archivo_desync)
    
    # Obtener el primer y último registro de ambos archivos
    primer_registro_sync = diccionario_sync[min(diccionario_sync.keys())]
    ultimo_registro_sync = diccionario_sync[max(diccionario_sync.keys())]
    primer_registro_desync = diccionario_desync[min(diccionario_desync.keys())]
    ultimo_registro_desync = diccionario_desync[max(diccionario_desync.keys())]

    # Calcular las diferencias de tiempo (rango) en segundos
    rango_sync = timestamp_a_segundos(ultimo_registro_sync['f_tiempo']) - timestamp_a_segundos(primer_registro_sync['o_tiempo'])
    rango_desync = timestamp_a_segundos(ultimo_registro_desync['f_tiempo']) - timestamp_a_segundos(primer_registro_desync['o_tiempo'])

    # Calcular el ajuste de tiempo entre los primeros registros
    ajuste = timestamp_a_segundos(primer_registro_sync['o_tiempo']) - timestamp_a_segundos(primer_registro_desync['o_tiempo'])

    # Calcular el ratio de desincronización
    ratio = rango_sync / rango_desync if rango_desync != 0 else 0

    # Ajustar los tiempos del archivo desincronizado
    for nro_linea, datos in diccionario_desync.items():
        # Obtener los tiempos de inicio y fin del registro actual
        o_tiempo_segundos = timestamp_a_segundos(datos['o_tiempo'])
        f_tiempo_segundos = timestamp_a_segundos(datos['f_tiempo'])

        # Calcular el ajuste
        registro_inicial = min(diccionario_desync.keys())
        try:
            ajuste_total = ajuste * (ratio * (rango_desync / (nro_linea - registro_inicial))) if (nro_linea - registro_inicial) != 0 else 0
        except ZeroDivisionError:
            ajuste_total = 0

        # Ajustar los tiempos
        nuevo_o_tiempo = o_tiempo_segundos - ajuste_total
        nuevo_f_tiempo = f_tiempo_segundos - ajuste_total

        # Convertir de vuelta a horas: minutos: segundos
        nuevo_o_tiempo_str = str(pd.to_timedelta(nuevo_o_tiempo, unit='s'))
        nuevo_f_tiempo_str = str(pd.to_timedelta(nuevo_f_tiempo, unit='s'))

        # Actualizar los tiempos en el diccionario
        diccionario_desync[nro_linea]['o_tiempo'] = nuevo_o_tiempo_str
        diccionario_desync[nro_linea]['f_tiempo'] = nuevo_f_tiempo_str

    # Devolver el diccionario sincronizado
    return diccionario_desync

def guardar_srt(diccionario, archivo_salida):
    with open(archivo_salida, 'w', encoding='latin-1') as file:
        for nro_linea, datos in diccionario.items():
            # Guardamos los tiempos tal cual, pero reemplazamos el punto por coma
            o_tiempo = datos['o_tiempo'].replace("0 days ", "").replace('.', ',')
            f_tiempo = datos['f_tiempo'].replace("0 days ", "").replace('.', ',')
            # Escribir en el archivo .srt en el formato requerido
            file.write(f"{nro_linea}\n")
            file.write(f"{o_tiempo} --> {f_tiempo}\n")
            file.write(f"{datos['texto']}\n\n")




Primer índice (1): {'o_tiempo': '0 days 00:01:00.360000', 'f_tiempo': '0 days 00:01:02.726000', 'texto': '<i>Yo conocï¿½ a Harvey Dent.</i>'}
Último índice (1880): {'o_tiempo': '0 days 02:35:08.906312029', 'f_tiempo': '0 days 02:35:11.739312029', 'texto': 'No puede haber un collar de perlas'}


In [ ]:

# Sincronizar los archivos
diccionario_sincronizado = sincronizar_srt(archivo_sincronizado, archivo_desincronizado)


# Guardar el nuevo archivo sincronizado

guardar_srt(diccionario_sincronizado, archivo_salida)
print(f"Archivo SRT guardado como {archivo_salida}")

Archivo SRT guardado como archivo_synced.srt
